In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import datetime
from datetime import timedelta

In [3]:
pd.options.mode.chained_assignment = None 

# 1. Get to know datasets

Olist, found in 2014, is the largest department store in Brazilian marketplaces. Olist connects small businesses from all over Brazil to channels without hassle and with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners.

After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.

In [4]:
# all_files = os.listdir('brazilian-ecommerce')
# csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))

map_files = {"orders": "olist_orders_dataset",
            "customers": "olist_customers_dataset",
            "sellers": "olist_sellers_dataset",
            "order_items": "olist_order_items_dataset",
            "products": "olist_products_dataset",
            "category": "product_category_name_translation",
            "payments": "olist_order_payments_dataset",
            "reviews": "olist_order_reviews_dataset",
            "geolocation": "olist_geolocation_dataset"}

for name in map_files.keys():
    file_path = 'brazilian-ecommerce/' + map_files[name] + '.csv'
    locals()[name] = pd.read_csv(file_path)

tables = map_files.keys()
df_schema = pd.DataFrame([globals()[i].columns for i in tables]).T
df_schema.columns = tables
df_schema

,orders,customers,sellers,order_items,products,category,payments,reviews,geolocation
0,order_id,customer_id,seller_id,order_id,product_id,product_category_name,order_id,review_id,geolocation_zip_code_prefix
1,customer_id,customer_unique_id,seller_zip_code_prefix,order_item_id,product_category_name,product_category_name_english,payment_sequential,order_id,geolocation_lat
2,order_status,customer_zip_code_prefix,seller_city,product_id,product_name_lenght,None,payment_type,review_score,geolocation_lng
3,order_purchase_timestamp,customer_city,seller_state,seller_id,product_description_lenght,None,payment_installments,review_comment_title,geolocation_city
4,order_approved_at,customer_state,None,shipping_limit_date,product_photos_qty,None,payment_value,review_comment_message,geolocation_state
5,order_delivered_carrier_date,None,None,price,product_weight_g,None,None,review_creation_date,None
6,order_delivered_customer_date,None,None,freight_value,product_length_cm,None,None,review_answer_timestamp,None
7,order_estimated_delivery_date,None,None,None,product_height_cm,None,None,None,None
8,None,None,None,None,product_width_cm,None,None,None,None


In [5]:
def get_info(df, table_name):
    df1 = pd.DataFrame([df.nunique(), df.dtypes, df.isna().sum()]).T
    df1.columns = ['unique_values', 'dtypes', 'nan_values']
    df1['total_records'] = len(df)
    df1['table'] = table_name
    df1 = df1.reset_index().rename(columns={'index': 'fields'})
    df1 = df1.set_index('table')
    return df1

df_info = pd.DataFrame()
for i in tables:
    table = globals()[i]
    df = get_info(table, i)
    df_info = df_info.append(df)

df_info = df_info.reset_index()

In [6]:
dup_fields = df_info[df_info.duplicated('fields',keep=False)]
rows = dup_fields['fields'].to_list()

def highlight(x):
    if x in rows:
        color = f'background: darkgreen'
    elif 'shipping' in x or 'deliver' in x:
        color = f'background-color : darkblue'
    else: 
        color = ''
    return color
df_info_format = df_info.style.applymap(highlight, subset=['fields'])

In [7]:
df_info_format

,table,fields,unique_values,dtypes,nan_values,total_records
0,orders,order_id,99441,object,0,99441
1,orders,customer_id,99441,object,0,99441
2,orders,order_status,8,object,0,99441
3,orders,order_purchase_timestamp,98875,object,0,99441
4,orders,order_approved_at,90733,object,160,99441
5,orders,order_delivered_carrier_date,81018,object,1783,99441
6,orders,order_delivered_customer_date,95664,object,2965,99441
7,orders,order_estimated_delivery_date,459,object,0,99441
8,customers,customer_id,99441,object,0,99441
9,customers,customer_unique_id,96096,object,0,99441


Data Schema Image

<img src=https://i.imgur.com/HRhd2Y0.png width="700">


# 2. Joining Datasets

## Geolocation

In [9]:
geolocation = geolocation.rename(columns={"geolocation_zip_code_prefix": "zip_code_prefix"})
sellers = sellers.rename(columns={"seller_zip_code_prefix": "zip_code_prefix"})
customers = customers.rename(columns={"customer_zip_code_prefix": "zip_code_prefix"})

sellers = pd.merge(sellers, geolocation[['zip_code_prefix','geolocation_lat','geolocation_lng']], on ='zip_code_prefix')
sellers = sellers.drop_duplicates(subset ="seller_id", keep = 'last') 

customers = pd.merge(customers, geolocation[['zip_code_prefix','geolocation_lat','geolocation_lng']], on ='zip_code_prefix')
customers = customers.drop_duplicates(subset ="customer_id", keep = 'last') 


## Customers & Sellers & Orders

In [562]:
print('orders: ', orders.shape)
print('customers: ', customers.shape)
# Joining 'delivered_orders' and 'customers' -> 'orders'
orders = pd.merge(orders, customers, on ='customer_id', how='left')
print('orders: ', orders.shape)

print('-------------')

print('order_items: ', order_items.shape)
print('sellers: ', sellers.shape)
# Joining 'sellers' and 'order_items' -> 'item_seller'
item_seller = pd.merge(order_items, sellers, on ='seller_id', how='left')
print('item_seller: ', item_seller.shape)

orders:  (99441, 8)
customers:  (99441, 5)
orders:  (99441, 12)
-------------
order_items:  (112650, 7)
sellers:  (3095, 4)
item_seller:  (112650, 10)


In [563]:
#Joining 'orders' and 'item_seller' -> 'delivered_orders'
print('orders: ', orders.shape)
print('item_seller: ', item_seller.shape)
orders = pd.merge(orders, item_seller, on ='order_id', how='left')
print('orders: ', orders.shape)
print('----------------')
orders = orders.rename(columns={"zip_code_prefix_x": "zip_code_prefix_customer",
                                                    "zip_code_prefix_y": "zip_code_prefix_seller",
                                                    "geolocation_lat_x": "geolocation_lat_customer",
                                                    "geolocation_lat_y": "geolocation_lat_seller",
                                                    "geolocation_lng_x": "geolocation_lng_customer",
                                                    "geolocation_lng_y": "geolocation_lng_seller"})


#Joining 'orders' and 'reviews'
reviews = reviews.groupby(['order_id'])['review_score'].median().reset_index(name='median_review_score').reset_index(drop=True)
print('reviews: ', reviews.shape)
orders = pd.merge(orders, reviews[['order_id','median_review_score']], on ='order_id', how = 'left')
print('orders: ', orders.shape)
print('----------------')


orders:  (99441, 12)
item_seller:  (112650, 10)
orders:  (113425, 21)
----------------
reviews:  (99441, 2)
orders:  (113425, 22)
----------------


In [564]:
#Joining 'products' and 'category' -> 'product_cat'
print('products: ', products.shape)
print('category: ', category.shape)
product_cat = pd.merge(products[['product_id', 'product_category_name']], category, on ='product_category_name', how='left')
print('product_cat: ', product_cat.shape)
print('-------------')

#Joining 'orders' and 'product_cat'
print('orders: ', orders.shape)
print('product_cat: ', product_cat.shape)
orders = pd.merge(orders, product_cat[['product_id', 'product_category_name_english']], on ='product_id', how='left')
print('orders: ', orders.shape)


products:  (32951, 9)
category:  (71, 2)
product_cat:  (32951, 3)
-------------
orders:  (113425, 22)
product_cat:  (32951, 3)
orders:  (113425, 23)


# 3. Data Cleaning

##  Date/Time

### Orders

In [565]:
def check_logic_datetime(df):
    print('delivered_customer < purchase: ')
    print((df['order_delivered_customer_date'] < df['order_purchase_timestamp']).value_counts())
    print('----------')

    print('delivered_customer < approved: ')
    print((df['order_delivered_customer_date'] < df['order_approved_at']).value_counts())
    print('----------')

    print('delivered_customer < delivered_carrier: ')
    print((df['order_delivered_customer_date'] < df['order_delivered_carrier_date']).value_counts())
    print('----------')

    print('delivered_carrier < purchase: ')
    print((df['order_delivered_carrier_date'] < df['order_purchase_timestamp']).value_counts())
    print('----------')

    print('delivered_carrier < approved: ')
    print((df['order_delivered_carrier_date'] < df['order_approved_at']).value_counts())
    print('----------')

    print('approved < purchase: ')
    print((df['order_approved_at'] < df['order_purchase_timestamp']).value_counts())
    print('----------')

In [566]:
def get_min_max_date(df, datetime_fields):
    datetime_info = df.filter(datetime_fields, axis=1)
    df1 = pd.DataFrame([datetime_info.min(), datetime_info.max()]).T
    df1.columns = ['min_date', 'max_date']
    return df1

In [567]:
# Convert fields into DateTime Type
def convert_to_datetime(df, fields):
    for field in fields:
        df[field] = pd.to_datetime(df[field])

# Convert fields into DateTime Type
datetime_fields = ['order_purchase_timestamp',
                    'order_approved_at', 
                    'order_delivered_carrier_date',
                    'order_delivered_customer_date',
                    'order_estimated_delivery_date']

convert_to_datetime(orders, datetime_fields)
get_min_max_date(orders, datetime_fields)

,min_date,max_date
order_purchase_timestamp,2016-09-04 21:15:19,2018-10-17 17:30:18
order_approved_at,2016-09-15 12:16:38,2018-09-03 17:40:06
order_delivered_carrier_date,2016-10-08 10:34:01,2018-09-11 19:48:28
order_delivered_customer_date,2016-10-11 13:46:32,2018-10-17 13:22:46
order_estimated_delivery_date,2016-09-30 00:00:00,2018-11-12 00:00:00


- status = 'created' -> chỉ có purchase_date và estimate_date
- status = 'approved' -> chỉ có purchase_date, approved_at và estimate_date
- status = 'invoiced' -> chỉ có purchase_date, approved_at và estimate_date
- status = 'processing' -> chỉ có purchase_date, approved_at và estimate_date
- status = 'unavailable' -> chỉ có purchase_date, approved_at và estimate_date
- status = 'canceled' -> chỉ có purchase_date, approved_at và estimate_date và có thể có carrier_date, customer_date
- status = 'shipped' -> chỉ có purchase_date, approved_at, carrier_date và estimate_date
- status = 'delivered' -> có full date

=> 1 order_id chỉ có 1 status

- approved_at null khi status in ['canceled', 'delivered', 'created']
- carrier_date null khi status in ['invoiced', 'processing', 'unavailable', 'canceled', 'created', 'approved', 'delivered']
- customer_date null khi status in ['invoiced', 'shipped', 'processing', 'unavailable', 'canceled', 'delivered', 'created', 'approved']
- estimate_date không bao giờ null

In [568]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 161
order_delivered_carrier_date     1968
order_delivered_customer_date    3229
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
order_item_id                     775
product_id                        775
seller_id                         775
shipping_limit_date               775
price                             775
freight_value                     775
seller_zip_code_prefix            775
seller_city                       775
seller_state                      775
median_review_score                 0
product_category_name_english    2402
dtype: int64

In [569]:
check_logic_datetime(orders)

delivered_customer < purchase: 
False    113425
dtype: int64
----------
delivered_customer < approved: 
False    113356
True         69
dtype: int64
----------
delivered_customer < delivered_carrier: 
False    113375
True         50
dtype: int64
----------
delivered_carrier < purchase: 
False    113238
True        187
dtype: int64
----------
delivered_carrier < approved: 
False    111874
True       1551
dtype: int64
----------
approved < purchase: 
False    113425
dtype: int64
----------


Có trường hợp ngày vận chuyển cho KH < ngày approve đơn

Có trường hợp ngày vận chuyển KH < ngày nhận hàng từ carrier

Có trường hợp ngày nhận hàng từ carrier < ngày lên đơn

Bảng order có thể dùng để xem lượng nhu cầu đặt hàng của customers dựa trên order_purchase_timestamp và order_status, nên bảng order có thể sử dụng mà chưa cần phải quan tâm đến tính hợp lí của các ngày.

### Delivered Orders

Tiếp theo, do chúng ta quan tâm nhiều nhất là các đơn đã hoàn thành, nên bước tiếp theo sẽ tiến hành clean data cho các đơn hàng đã hoàn thành

In [570]:
# Splitting 'orders' into delivered and undelivered
undelivered_orders = orders.loc[orders['order_status'] != 'delivered']
delivered_orders = orders.loc[orders['order_status'] == 'delivered']

get_min_max_date(delivered_orders, datetime_fields)

,min_date,max_date
order_purchase_timestamp,2016-09-15 12:16:38,2018-08-29 15:00:37
order_approved_at,2016-09-15 12:16:38,2018-08-29 15:10:26
order_delivered_carrier_date,2016-10-08 10:34:01,2018-09-11 19:48:28
order_delivered_customer_date,2016-10-11 13:46:32,2018-10-17 13:22:46
order_estimated_delivery_date,2016-10-04 00:00:00,2018-10-25 00:00:00


In [571]:
delivered_orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_carrier_date        2
order_delivered_customer_date       8
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
price                               0
freight_value                       0
seller_zip_code_prefix              0
seller_city                         0
seller_state                        0
median_review_score                 0
product_category_name_english    1559
dtype: int64

In [572]:
# Clean order_approved_at
delivered_orders['order_approved_at'] = np.where(delivered_orders['order_approved_at'].isnull(), delivered_orders['order_purchase_timestamp'], delivered_orders['order_approved_at'])

# Clean days_delivered_carrier_after_approve
carrier_not_null = delivered_orders[delivered_orders['order_delivered_carrier_date'].isnull() == False]
carrier_not_null['days_delivered_carrier_after_approve'] = (carrier_not_null['order_delivered_carrier_date'] - carrier_not_null['order_approved_at']).dt.days

median_gap_days = carrier_not_null['days_delivered_carrier_after_approve'].median()
delivered_orders['order_delivered_carrier_date'] = np.where(delivered_orders['order_delivered_carrier_date'].isnull(), delivered_orders['order_approved_at'] + timedelta(days=median_gap_days), delivered_orders['order_delivered_carrier_date'])

# Clean days_delivered_customer_after_approve
customer_not_null = delivered_orders[delivered_orders['order_delivered_customer_date'].isnull() == False]
customer_not_null['days_estimate_delivered_customer'] = (customer_not_null['order_estimated_delivery_date'] - customer_not_null['order_delivered_customer_date']).dt.days

median_gap_days = customer_not_null['days_estimate_delivered_customer'].median()
delivered_orders['order_delivered_customer_date'] = np.where(delivered_orders['order_delivered_customer_date'].isnull(), delivered_orders['order_estimated_delivery_date'] - timedelta(days=median_gap_days), delivered_orders['order_delivered_customer_date'])

In [573]:
check_logic_datetime(delivered_orders)

delivered_customer < purchase: 
False    110197
dtype: int64
----------
delivered_customer < approved: 
False    110128
True         69
dtype: int64
----------
delivered_customer < delivered_carrier: 
False    110147
True         50
dtype: int64
----------
delivered_carrier < purchase: 
False    110011
True        186
dtype: int64
----------
delivered_carrier < approved: 
False    108655
True       1542
dtype: int64
----------
approved < purchase: 
False    110197
dtype: int64
----------


Đóng vai người mua hàng, chúng ta sẽ quan tâm số ngày khách hàng nhận hàng sau khi đã đặt hàng => outbound_delivery_lead_date

Ngoài ra, chúng ta cần theo dõi số ngày mà carrier/merchant gửi hàng đến cho mình sau khi khách đã đặt đơn => inbound_delivery_lead_date. 

Do có 165 case delivered_carrier < purchase_date và chưa hiểu rõ business model cụ thể như thế nào, bài phân tích này giả định delivered_carried bắt buộc phải xảy ra sau khi khách hàng đã lên đơn => 165 delivered_carrier_date đó được clean bằng cách cho bằng purchase_date.

In [574]:
orders.to_csv('orders.csv', index=False)
delivered_orders.to_csv('delivered_orders.csv', index=False)

## Orders and Order_items

In [225]:
order_items.groupby('order_id').size().reset_index().sort_values(0, ascending=False)

,order_id,0
50137,8272b63d03f5f79c56e9e4120aec44ef,21
10459,1b15974a0141d54e36626dca3fdc731a,20
65715,ab14fdcfbe524636d65ee38360e22ce8,20
60941,9ef13efd6949e4573a18964dd1bbe7f5,15
25583,428a2f660dc84138d969ccd69a0ab6d5,15
...,...,...
34640,5a0911d70c1f85d3bed0df1bf693a6dd,1
34639,5a082b558a3798d3e36d93bfa8ca1eae,1
34637,5a07264682e0b8fbb3f166edbbffc6e8,1
34636,5a071192a28951b76774e5a760c8c9b7,1


- freight_value is the shipping value from seller to customer. 1 product_id can have different freight_values, maybe depends on the distance.
- price is the price of the product. 1 product_id can have different price value, maybe depends on the period of time (although from the same seller_id)
- 1 product_id can have multiple seller_ids
- 1 product_id in 1 order can be delivered by 1 seller_id only (no mix sellers for 1 product in an order)
